<a href="https://colab.research.google.com/github/gustavesg/TCC/blob/main/2_Analise_Explorat%C3%B3ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importações**

In [ ]:
#libraries import
#importa bibliotecas

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu
from scipy.stats import kstest
from scipy.stats import shapiro
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage


#drive import
#importa o drive

from google.colab import drive
drive.mount('/content/drive')

#imports database
#importa base de dados

df_tratada_treino  = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_tratados_treino/loan_tratada_treino.csv')
df_tratada_teste  = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_tratados_teste/loan_tratada_teste.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#substitui nulos pela mediana dos dados
df_tratada_treino[df_tratada_treino.select_dtypes(include='number').columns] = df_tratada_treino.select_dtypes(include='number').fillna(df_tratada_treino.median())

#retira coluna criada
df_tratada_treino.drop(columns = ['earliest_cr_line_last_credit_pull_d_years'], axis = 1, inplace = True)
df_tratada_teste.drop(columns = ['earliest_cr_line_last_credit_pull_d_years'], axis = 1, inplace = True)


In [ ]:
df_tratada_treino.shape

(349999, 125)

### **Variaveis Altamente Correlacionadas**

In [ ]:
# limite para considerar duas variáveis altamente correlacionadas
threshold = 0.9

#calcula matriz de correlação
matriz_correlacao = df_tratada_treino.corr(method='pearson')

#pares de correlação
pares_correlacao = matriz_correlacao.unstack().reset_index()

#renomeia colunas
pares_correlacao.columns = ['Variável 1', 'Variável 2', 'Correlação']

#pares com alta correlação
high_corr_pairs = pares_correlacao[(pares_correlacao['Correlação'].abs() > threshold) & (pares_correlacao['Variável 1'] != pares_correlacao['Variável 2'])]

# converter a matriz de correlação para uma matriz de distância
distance_matrix = 1 - np.abs(matriz_correlacao)

# exclui valores infinitos e nulos
distance_matrix = distance_matrix.replace([np.inf, -np.inf], np.nan).fillna(0)

# A matriz de distância precisa ser convertida para um formato de vetor que o squareform aceita
distance_matrix_condensed = squareform(distance_matrix.values, checks=False)


# Linkage algoritmo de agrupamento hierárquico novamente na matriz de distância
Z = linkage(squareform(distance_matrix_condensed), 'ward')

# Formar grupos com base no threshold definido
clusters = fcluster(Z, threshold, criterion='distance')

#  Mapear as variáveis para seus respectivos grupos
groups = {i: [] for i in np.unique(clusters)}
for i, var in enumerate(matriz_correlacao.columns):
    groups[clusters[i]].append(var)

variaveis_excluidas = []

contador = 0
# Exibir os grupos formados
for group, variables in groups.items():

    #grupos com mais de duas variaveis correlacionadas
    if len(variables) >= 2:

      print(f'Grupo {group}: {variables}')

      contador = contador + 1

      for  i in range(0,len(variables)):

          variaveis_excluidas.append(variables[i])



Grupo 1: ['out_prncp', 'out_prncp_inv', 'policy_code', 'deferral_term', 'hardship_length']
Grupo 2: ['application_type', 'verification_status_joint']
Grupo 4: ['annual_inc_joint', 'revol_bal_joint']
Grupo 6: ['sec_app_open_acc', 'sec_app_num_rev_accts']
Grupo 9: ['hardship_amount', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount']
Grupo 11: ['mths_since_last_delinq', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq']
Grupo 12: ['num_accts_ever_120_pd', 'pct_tl_nvr_dlq']
Grupo 13: ['delinq_2yrs', 'num_tl_90g_dpd_24m']
Grupo 15: ['pub_rec', 'pub_rec_bankruptcies']
Grupo 18: ['acc_now_delinq', 'num_tl_30dpd']
Grupo 19: ['sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med']
Grupo 39: ['mo_sin_rcnt_rev_tl_op', 'mths_since_recent_bc']
Grupo 41: ['acc_open_past_24mths', 'num_tl_op_past_12m']
Grupo 42: ['open_rv_12m', 'open_rv_24m']
Grupo 44: ['total_bal_ex_mort', 'total_il_high_credit_limit']
Grupo 45: ['open_act_il', 'total_bal_il'

<ipython-input-4-7ac0fca4241b>:27: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(squareform(distance_matrix_condensed), 'ward')


In [ ]:
contador

32

### **Escolha de 1 variável de cada cluster para manter**

In [ ]:
variaveis_mantidas = [
                      'out_prncp','application_type',
                      'annual_inc_joint',
                      'sec_app_open_acc',
                      'hardship_amount',
                      'mths_since_last_delinq',
                      'num_accts_ever_120_pd',
                      'delinq_2yrs',
                      'pub_rec',
                      'acc_now_delinq',
                      'sec_app_chargeoff_within_12_mths',
                      'mths_since_recent_bc',
                      'acc_open_past_24mths',
                      'open_rv_24m',
                      'total_il_high_credit_limit',
                      'total_bal_il',
                      'mths_since_recent_inq',
                      'inq_last_12m',
                      'open_il_24m',
                      'total_bc_limit',
                      'tot_cur_bal',
                      'earliest_cr_line_years',
                      'debt_settlement_flag',
                      'recoveries',
                      'int_rate',
                      'bc_util',
                      'loan_amnt',
                      'total_pymnt',
                      'open_acc',
                      'num_bc_tl',
                      'num_actv_bc_tl',
                      'num_op_rev_tl'
                      ]

#variaveis para excluir da base
variaveis_pra_exclusao = [item for item in variaveis_excluidas if item not in variaveis_mantidas]

df_tratada_treino = df_tratada_treino.drop(columns = variaveis_pra_exclusao)
df_tratada_teste = df_tratada_teste.drop(columns = variaveis_pra_exclusao)

### **Exclusão de Variáveis a Posteriori**

In [ ]:
#algumas variáveis são conhecidas apenas após final ou durante o andamento do contrato, para fins de nossa
#análise precisamos exclui-las

exclusao_posteriori =  [

                        'total_pymnt',
                        'total_rec_int',
                        'total_rec_late_fee',
                        'recoveries',
                        'last_pymnt_amnt',
                        'tot_coll_amt',
                        'chargeoff_within_12_mths',
                        'hardship_amount',
                        'hardship_dpd',
                        'hardship_last_payment_amount',
                        'debt_settlement_flag',
                        'settlement_amount',
                        'settlement_percentage',
                        'settlement_term'

                        ]


#exclui variáveis dos datasets
df_tratada_treino = df_tratada_treino.drop(columns = exclusao_posteriori)
df_tratada_teste = df_tratada_teste.drop(columns = exclusao_posteriori)


In [ ]:
df_tratada_treino.shape

(349999, 68)

### **Floresta Aleatória (Importância das Features)**

In [ ]:
# declara as features e a target loan_status
X = df_tratada_treino.drop(columns = ['loan_status'])
y = df_tratada_treino['loan_status']

#nome das features
nome_features = list(X.columns)

#tratamento: substituir valores nulos pela média da coluna
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

#classificador Random Forest
rf = RandomForestClassifier()

# Treina classificador
rf.fit(X_imputed, y)

#importância das features
importances = rf.feature_importances_

# Ordene as importâncias em ordem decrescente
indices = np.argsort(importances)[::-1]

# Imprima as importâncias das features
print("Importância das features:")
for i, feature in enumerate(indices):
    print(f"{i+1}. Feature {nome_features[feature]}: {importances[feature]}")



Importância das features:
1. Feature int_rate: 0.08547652071726677
2. Feature dti: 0.04361497354232057
3. Feature bc_open_to_buy: 0.03668037149111902
4. Feature avg_cur_bal: 0.03577764003580934
5. Feature annual_inc: 0.035619773142192494
6. Feature revol_bal: 0.035548988191024054
7. Feature loan_amnt: 0.03527535009083013
8. Feature bc_util: 0.03397936428742329
9. Feature tot_cur_bal: 0.03373445309687637
10. Feature total_bc_limit: 0.03307618761855206
11. Feature mo_sin_old_il_acct: 0.031569463536882024
12. Feature total_il_high_credit_limit: 0.028928691508718976
13. Feature mths_since_recent_bc: 0.02875934889073542
14. Feature total_acc: 0.028446375842617024
15. Feature term: 0.0246660531939444
16. Feature mths_since_recent_inq: 0.023466132134636657
17. Feature open_acc: 0.02270405424821959
18. Feature mths_since_last_delinq: 0.02264274740193948
19. Feature mo_sin_rcnt_tl: 0.022580981685315554
20. Feature num_il_tl: 0.022456769194089638
21. Feature acc_open_past_24mths: 0.0217004032689

### **Seleção variáveis mais importantes**

In [ ]:
df_tratada_treino = df_tratada_treino[['int_rate','dti', 'bc_open_to_buy',
                                        'avg_cur_bal',
                                        'revol_bal',
                                        'annual_inc',
                                        'loan_amnt',
                                        'tot_cur_bal',
                                        'bc_util',
                                        'total_bc_limit',
                                        'loan_status'

                                        ]]

df_tratada_teste = df_tratada_teste[[   'int_rate','dti',
                                        'bc_open_to_buy',
                                        'avg_cur_bal',
                                        'revol_bal',
                                        'annual_inc',
                                        'loan_amnt',
                                        'tot_cur_bal',
                                        'bc_util',
                                        'total_bc_limit',
                                        'loan_status'

                                        ]]

### **Remoção de Outliers**

In [ ]:
def remove_outliers_iqr_all_columns(data):

    # Para cada coluna numérica, calcula os limites IQR e filtra os outliers
    for column in data.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR

        # Filtra os dados para manter apenas valores dentro dos limites para cada coluna
        data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

    return data

In [ ]:
df_tratada_sem_out_treino = remove_outliers_iqr_all_columns(df_tratada_treino)
df_tratada_sem_out_teste = remove_outliers_iqr_all_columns(df_tratada_teste)


In [ ]:
df_tratada_sem_out_treino

,int_rate,dti,bc_open_to_buy,avg_cur_bal,revol_bal,annual_inc,loan_amnt,tot_cur_bal,bc_util,total_bc_limit,loan_status
0,15.59,17.18,9484.0,21694.0,26155,120000.0,30000,412179.0,56.9,22000.0,1
2,13.49,26.83,3288.0,9540.0,34029,103000.0,22000,286208.0,77.0,26300.0,1
3,11.14,18.00,10199.0,989.0,13851,35000.0,11500,13851.0,57.3,23900.0,1
5,12.29,7.57,5611.0,34419.0,8989,162000.0,30000,206516.0,61.6,14600.0,1
7,10.75,18.39,13218.0,27635.0,14306,50000.0,16000,221076.0,29.3,18700.0,1
...,...,...,...,...,...,...,...,...,...,...,...
349992,10.99,8.12,12591.0,14121.0,13110,55000.0,9600,112968.0,34.4,19200.0,0
349993,14.49,35.43,229.0,3957.0,15651,52000.0,15000,51441.0,98.1,12000.0,0
349995,19.05,6.92,287.0,1060.0,10602,74000.0,16000,10602.0,95.6,6500.0,0
349996,25.99,29.83,8009.0,4896.0,17574,85000.0,16000,97915.0,63.8,22100.0,0


### **Normalização Min Max dos dados**

In [ ]:
df_tratada_sem_out_treino_min_max = (df_tratada_sem_out_treino - df_tratada_sem_out_treino.min()) / (df_tratada_sem_out_treino.max() - df_tratada_sem_out_treino.min())
df_tratada_sem_out_teste_min_max = (df_tratada_sem_out_teste - df_tratada_sem_out_teste.min()) / (df_tratada_sem_out_teste.max() - df_tratada_sem_out_teste.min())

In [ ]:
df_tratada_sem_out_treino_min_max.min()


,0
int_rate,0.0
dti,0.0
bc_open_to_buy,0.0
avg_cur_bal,0.0
revol_bal,0.0
annual_inc,0.0
loan_amnt,0.0
tot_cur_bal,0.0
bc_util,0.0
total_bc_limit,0.0


###**Hipótese de normalidade dos dados**

In [ ]:
# Nível de significância
alpha = 0.05

for column in df_tratada_sem_out_treino_min_max.columns:

    # Valores da variável para bons e maus pagadores
    values_good = df_tratada_sem_out_treino_min_max[df_tratada_sem_out_treino_min_max['loan_status'] == 1][column].dropna()
    values_bad = df_tratada_sem_out_treino_min_max[df_tratada_sem_out_treino_min_max['loan_status'] == 0][column].dropna()

    # Testando se os dados seguem uma distribuição normal com Shapiro-Wilk
    stat_good, p_value_good = shapiro(values_good)
    stat_bad, p_value_bad = shapiro(values_bad)

    # Impressão
    print(104*'=')
    print(f'Análise para a coluna: {column}')
    print(104*'=')

    print(104*'=')
    print('= '*20 + 'Amostra de bons pagadores' + ' ='*20)
    print(104*'=')
    print('Estatística do teste:', stat_good)
    print('Valor-p:', p_value_good)

    # Se o valor-p for menor que alpha, rejeita a hipótese nula
    if p_value_good < alpha:
        print('Rejeita a hipótese nula: os dados não seguem uma distribuição normal')
    else:
        print('Falha em rejeitar a hipótese nula: os dados seguem uma distribuição normal')

    # Impressão
    print(104*'=')
    print('= '*20 + 'Amostra de maus pagadores' + ' ='*20)
    print(104*'=')
    print('Estatística do teste:', stat_bad)
    print('Valor-p:', p_value_bad)

    # Se o valor-p for menor que alpha, rejeita a hipótese nula
    if p_value_bad < alpha:
        print('Rejeita a hipótese nula: os dados não seguem uma distribuição normal')
    else:
        print('Falha em rejeitar a hipótese nula: os dados seguem uma distribuição normal')
    print(104*'=')




/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 154869.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 162711.
  res = hypotest_fun_out(*samples, **kwds)


Análise para a coluna: int_rate
= = = = = = = = = = = = = = = = = = = = Amostra de bons pagadores = = = = = = = = = = = = = = = = = = = =
Estatística do teste: 0.9648202781366032
Valor-p: 1.4949763398577161e-93
Rejeita a hipótese nula: os dados não seguem uma distribuição normal
= = = = = = = = = = = = = = = = = = = = Amostra de maus pagadores = = = = = = = = = = = = = = = = = = = =
Estatística do teste: 0.977310672939488
Valor-p: 1.0650972771591878e-83
Rejeita a hipótese nula: os dados não seguem uma distribuição normal
Análise para a coluna: dti
= = = = = = = = = = = = = = = = = = = = Amostra de bons pagadores = = = = = = = = = = = = = = = = = = = =
Estatística do teste: 0.9886188873387337
Valor-p: 2.143788772992201e-67
Rejeita a hipótese nula: os dados não seguem uma distribuição normal
= = = = = = = = = = = = = = = = = = = = Amostra de maus pagadores = = = = = = = = = = = = = = = = = = = =
Estatística do teste: 0.9933669716059021
Valor-p: 4.225794691729679e-57
Rejeita a hipótese nu

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


### **Salvamento das bases**

In [ ]:
df_tratada_sem_out_treino_min_max.to_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_treino/base_final_treino')
df_tratada_sem_out_treino.to_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_treino/base_no_normalizada_treino')

df_tratada_sem_out_teste_min_max.to_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_teste/base_final_teste')
df_tratada_sem_out_teste.to_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_teste/base_no_normalizada_teste')
